In [264]:
import pandas as pd
import numpy as np

### nombres

In [265]:
header_file = 'header_info_images4'
data_file = 'V3V_lc52.data'
otuput_name = 'V3V_lc52_HJD.data'

### archivos 

In [266]:
# cargamos archivos
header_info = pd.read_csv(header_file, delim_whitespace=True, 
                         names=['file', 'JD', 'HJD', 'UT', 'EXPTIME', 'FILTER'])

lc_data = pd.read_csv(data_file, delim_whitespace=True,
                     names=['Tiempo(JD)', 'Flujo', 'errFlujo', 'refFlujo', 'errRefF', 'calSNR'])

# JD - 2460400
header_info['JD'] = header_info['JD'] - 2460400
header_info['HJD'] = header_info['HJD'] - 2460400  # Si también quieres ajustar HJD

In [267]:
header_info.head()

,file,JD,HJD,UT,EXPTIME,FILTER
0,202405010178o.fits,31.831055,31.836794,07:55:43.186945,120.0,V
1,202405010180o.fits,31.833791,31.839529,07:59:39.501064,120.0,V
2,202405010182o.fits,31.836491,31.842230,08:03:32.862361,120.0,V
3,202405010184o.fits,31.839226,31.844964,08:07:29.114749,120.0,V
4,202405010186o.fits,31.841925,31.847663,08:11:22.343940,120.0,V


In [268]:
lc_data.head()

,Tiempo(JD),Flujo,errFlujo,refFlujo,errRefF,calSNR
0,31.831055,-2781.7134,207.0539,-2758.7772,228.5742,0.7966
1,31.833791,-2627.1430,205.5263,-2635.7390,227.1235,0.8796
2,31.836491,-2469.8416,207.8666,-2495.9483,222.2319,0.8208
3,31.839226,-2214.1338,207.2856,-2209.2018,226.6226,0.8165
4,31.841925,-2434.7003,208.2351,-2170.0728,227.2063,0.8127


### offset

In [269]:
# calculamos el offset necesario para quitarnos negativos
offset_obj = abs(lc_data['Flujo'].min()) + 10
offset_ref = abs(lc_data['refFlujo'].min()) + 10

# el maximo de los dos
offset = max(offset_obj, offset_ref)
print(offset)

23961.5746


In [270]:
# aplicamos el offset 
lc_data['Flujo'] = lc_data['Flujo'] + offset
lc_data['refFlujo'] = lc_data['refFlujo'] + offset

In [271]:
# comprobamos 
print(lc_data['Flujo'].min())
print(lc_data['refFlujo'].min())

458.6054000000004
10.0


### Magnitud

In [272]:
# Flujo a magnitud instrumental
exptime = 120 # la sacamos del header_info_images4

lc_data['Flujo'] = 25.0 - 2.5*np.log10(lc_data['Flujo']) + 2.5*np.log10(exptime)
lc_data['refFlujo'] = 25.0 - 2.5*np.log10(lc_data['refFlujo']) + 2.5*np.log10(exptime)

In [273]:
header_info.head()

,file,JD,HJD,UT,EXPTIME,FILTER
0,202405010178o.fits,31.831055,31.836794,07:55:43.186945,120.0,V
1,202405010180o.fits,31.833791,31.839529,07:59:39.501064,120.0,V
2,202405010182o.fits,31.836491,31.842230,08:03:32.862361,120.0,V
3,202405010184o.fits,31.839226,31.844964,08:07:29.114749,120.0,V
4,202405010186o.fits,31.841925,31.847663,08:11:22.343940,120.0,V


In [274]:
lc_data.head()

,Tiempo(JD),Flujo,errFlujo,refFlujo,errRefF,calSNR
0,31.831055,19.383145,207.0539,19.381970,228.5742,0.7966
1,31.833791,19.375250,205.5263,19.375688,227.1235,0.8796
2,31.836491,19.367275,207.8666,19.368594,222.2319,0.8208
3,31.839226,19.354433,207.2856,19.354187,226.6226,0.8165
4,31.841925,19.365501,208.2351,19.352235,227.2063,0.8127


### JD a HJD

In [275]:
zip(header_info['JD'], header_info['HJD'])[:5]

[(31.83105544000864, 31.836793589871377),
 (31.833790509961545, 31.839528650045395),
 (31.83649143995717, 31.842229560017586),
 (31.839225810021162, 31.844963919837028),
 (31.841925229877234, 31.847663320135325)]

In [276]:
# diccionario de JD → HJD
#jd_to_hjd = dict(zip(header_info['JD'], header_info['HJD']))

# diccionario de JD a HJD (con tolerancia para diferencias mínimas)
jd_to_hjd = {round(jd, 6): hjd for jd, hjd in zip(header_info['JD'], header_info['HJD'])}

In [277]:
#jd_to_hjd

In [278]:
# reemplazamos en lc_data
#lc_data['Tiempo(JD)'] = lc_data['Tiempo(JD)'].map(jd_to_hjd)

# reemplazar en lc_data (usando el JD redondeado a 6 decimales para matching)
lc_data['Tiempo(JD)'] = lc_data['Tiempo(JD)'].apply(
    lambda x: jd_to_hjd.get(round(x, 6), x)  # Mantiene el original si no hay match
)

In [279]:
lc_data.head()

,Tiempo(JD),Flujo,errFlujo,refFlujo,errRefF,calSNR
0,31.836794,19.383145,207.0539,19.381970,228.5742,0.7966
1,31.839529,19.375250,205.5263,19.375688,227.1235,0.8796
2,31.842230,19.367275,207.8666,19.368594,222.2319,0.8208
3,31.844964,19.354433,207.2856,19.354187,226.6226,0.8165
4,31.847663,19.365501,208.2351,19.352235,227.2063,0.8127


### guardar

In [280]:
# guardar 
lc_data.to_csv(otuput_name, sep='\t', index=False, header=False)